# 0. Setup Paths

In [1]:
import os

In [2]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [3]:
paths = {
    'WORKSPACE_PATH': os.path.join('..','Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('..','Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('..','Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('..','Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('..','Tensorflow', 'workspace','images'),
    'TRAIN_IMAGE_PATH': os.path.join('..','Tensorflow', 'workspace','images','train'),
    'TEST_IMAGE_PATH': os.path.join('..','Tensorflow', 'workspace','images','test'),
    'MODEL_PATH': os.path.join('..','Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('..','Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('..','Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('..','Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('..','Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('..','Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('..','Tensorflow','protoc')
 }

In [4]:
files = {
    'PIPELINE_CONFIG':os.path.join('..','Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [5]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

# 1. Download TF Models Pretrained Models from Tensorflow Model Zoo and Install TFOD

In [6]:
# https://www.tensorflow.org/install/source_windows

In [6]:
if os.name=='nt':
    !pip install wget
    import wget

  Using cached wget-3.2.zip (10 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for wget: started
  Running setup.py install for wget: finished with status 'done'


  DEPRECATION: wget is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559


In [10]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

Cloning into 'Tensorflow\models'...
Updating files:  80% (2570/3180)
Updating files:  81% (2576/3180)
Updating files:  82% (2608/3180)
Updating files:  83% (2640/3180)
Updating files:  84% (2672/3180)
Updating files:  85% (2703/3180)
Updating files:  86% (2735/3180)
Updating files:  87% (2767/3180)
Updating files:  88% (2799/3180)
Updating files:  89% (2831/3180)
Updating files:  90% (2862/3180)
Updating files:  91% (2894/3180)
Updating files:  92% (2926/3180)
Updating files:  93% (2958/3180)
Updating files:  94% (2990/3180)
Updating files:  95% (3021/3180)
Updating files:  96% (3053/3180)
Updating files:  97% (3085/3180)
Updating files:  98% (3117/3180)
Updating files:  99% (3149/3180)
Updating files: 100% (3180/3180)
Updating files: 100% (3180/3180), done.


In [7]:
# Install Tensorflow Object Detection 
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

100% [..........................................................................] 1468733 / 1468733        1 arquivo(s) movido(s).
        1 arquivo(s) copiado(s).
running build
running build_py
copying object_detection\protos\anchor_generator_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\argmax_matcher_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\bipartite_matcher_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\box_coder_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\box_predictor_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\calibration_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\center_net_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\eval_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\faster_rcnn_box_coder_pb2.py -> build\lib\object_detection\prot

G:\Users\gui20\python_projects\ANPR_FINAL\anprsys\lib\site-packages\setuptools\command\install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
G:\Users\gui20\python_projects\ANPR_FINAL\anprsys\lib\site-packages\setuptools\command\easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
zip_safe flag not set; analyzing archive contents...
object_detection.core.__pycache__.densepose_ops.cpython-310: module references __file__
object_detection.core.__pycache__.preprocessor.cpython-310: module MAY be using inspect.stack
object_detection.utils.__pycache__.autoaugment_utils.cpython-310: module MAY be using inspect.stack
G:\Users\gui20\python_projects\ANPR_FINAL\anprsys\lib\site-packages\pkg_resources\__init__.py:123: PkgResourcesDeprecationWarning:  is an invalid version and will not be supported in a futur

Obtaining file:///G:/Users/gui20/python_projects/ANPR_FINAL/Tensorflow/models/research/slim
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached tf_slim-1.1.0-py2.py3-none-any.whl (352 kB)
  Using cached absl_py-1.3.0-py3-none-any.whl (124 kB)
  Running setup.py develop for slim


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
object-detection 0.1 requires apache-beam, which is not installed.
object-detection 0.1 requires avro-python3, which is not installed.
object-detection 0.1 requires contextlib2, which is not installed.
object-detection 0.1 requires Cython, which is not installed.
object-detection 0.1 requires lxml, which is not installed.
object-detection 0.1 requires matplotlib, which is not installed.
object-detection 0.1 requires pillow, which is not installed.
object-detection 0.1 requires pycocotools, which is not installed.
tf-models-official 2.10.0 requires Cython, which is not installed.
tf-models-official 2.10.0 requires gin-config, which is not installed.
tf-models-official 2.10.0 requires google-api-python-client>=1.6.7, which is not installed.
tf-models-official 2.10.0 requires immutabledict, which is not installed.
tf

In [8]:
!pip install tensorflow --upgrade

  Using cached tensorflow-2.10.0-cp310-cp310-win_amd64.whl (455.9 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached wrapt-1.14.1-cp310-cp310-win_amd64.whl (35 kB)
  Using cached tensorflow_estimator-2.10.0-py2.py3-none-any.whl (438 kB)
  Using cached typing_extensions-4.4.0-py3-none-any.whl (26 kB)
  Using cached protobuf-3.19.6-cp310-cp310-win_amd64.whl (895 kB)
     ---------------------------------------- 3.6/3.6 MB 25.7 MB/s eta 0:00:00
  Using cached h5py-3.7.0-cp310-cp310-win_amd64.whl (2.6 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached numpy-1.23.4-cp310-cp310-win_amd64.whl (14.6 MB)
  Using cached tensorboard-2.10.1-py3-none-any.whl (5.9 MB)
  Using cached libclang-14.0.6-py2.py3-none-win_amd64.whl (14.2 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
  Using cached gast-0.4.0-py

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
object-detection 0.1 requires apache-beam, which is not installed.
object-detection 0.1 requires avro-python3, which is not installed.
object-detection 0.1 requires contextlib2, which is not installed.
object-detection 0.1 requires Cython, which is not installed.
object-detection 0.1 requires lxml, which is not installed.
object-detection 0.1 requires matplotlib, which is not installed.
object-detection 0.1 requires pillow, which is not installed.
object-detection 0.1 requires pycocotools, which is not installed.
sacrebleu 2.2.0 requires lxml, which is not installed.
sacrebleu 2.2.0 requires portalocker, which is not installed.
sacrebleu 2.2.0 requires regex, which is not installed.
sacrebleu 2.2.0 requires tabulate>=0.8.9, which is not installed.
tf-models-official 2.10.0 requires Cython, which is not installed.


In [9]:
!pip install tensorflow-gpu --upgrade

  Using cached tensorflow_gpu-2.10.0-cp310-cp310-win_amd64.whl (455.9 MB)


In [10]:
!pip install matplotlib --upgrade

     ---------------------------------------- 7.2/7.2 MB 38.5 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 39.6 MB/s eta 0:00:00
  Using cached kiwisolver-1.4.4-cp310-cp310-win_amd64.whl (55 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     ------------------------------------- 163.6/163.6 kB 10.2 MB/s eta 0:00:00
     ------------------------------------- 965.4/965.4 kB 63.7 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
object-detection 0.1 requires apache-beam, which is not installed.
object-detection 0.1 requires avro-python3, which is not installed.
object-detection 0.1 requires contextlib2, which is not installed.
object-detection 0.1 requires Cython, which is not installed.
object-detection 0.1 requires lxml, which is not installed.
object-detection 0.1 requires pycocotools, which is not installed.
tf-models-official 2.10.0 requires Cython, which is not installed.
tf-models-official 2.10.0 requires gin-config, which is not installed.
tf-models-official 2.10.0 requires google-api-python-client>=1.6.7, which is not installed.
tf-models-official 2.10.0 requires immutabledict, which is not installed.
tf-models-official 2.10.0 requires kaggle>=1.3.9, which is not installed.
tf-models-official 2.10.0 requires oauth2client, which i

In [11]:
!pip install pyaml --upgrade

  Using cached pyaml-21.10.1-py2.py3-none-any.whl (24 kB)
  Using cached PyYAML-6.0-cp310-cp310-win_amd64.whl (151 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-models-official 2.10.0 requires Cython, which is not installed.
tf-models-official 2.10.0 requires gin-config, which is not installed.
tf-models-official 2.10.0 requires google-api-python-client>=1.6.7, which is not installed.
tf-models-official 2.10.0 requires immutabledict, which is not installed.
tf-models-official 2.10.0 requires kaggle>=1.3.9, which is not installed.
tf-models-official 2.10.0 requires oauth2client, which is not installed.
tf-models-official 2.10.0 requires opencv-python-headless==4.5.2.52, which is not installed.
tf-models-official 2.10.0 requires py-cpuinfo>=3.3.0, which is not installed.
tf-models-official 2.10.0 requires pycocotools, which is not installed.
tf-models-official 2.10.0 requires sentencepiece, which is not installed.
tf-models-official 2.10.0 requires seqeval, which is not 

In [12]:
!pip install pytz --upgrade

     -------------------------------------- 498.1/498.1 kB 7.7 MB/s eta 0:00:00


In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

# 2. Define Train and Test Photos Dataset

In [18]:
photos = r'C:\Users\gui20\Desktop\midia_tcc\Dataset_2_Advanced\2_No_Segmented_Data\\'

In [19]:
data_list = []
for i in os.listdir(photos):
    if '.jpg' in i:
        data_list.append(int(i.replace('.jpg', '')))
data_list = sorted(data_list)

In [20]:
import random
random.seed(1112)
random.shuffle(data_list)

train_data = data_list[:round(len(data_list)*0.7)]
test_data = data_list[round(len(data_list)*0.7):]

In [21]:
len(train_data)*2

1798

In [22]:
len(test_data)*2

772

In [23]:
#treino
destiny = os.path.join('..','Tensorflow','workspace','images','train')
for i in train_data:
    img = str(i) + '.jpg'
    xml = str(i) + '.xml'
    
    command_jpg = 'copy {} {}'.format(photos + img, destiny + img)
    command_xml = 'copy {} {}'.format(photos + xml, destiny + xml)
    
    os.popen(command_jpg) 
    os.popen(command_xml) 

In [24]:
#teste
destiny = os.path.join('..','Tensorflow','workspace','images','test')
for i in test_data:
    img = str(i) + '.jpg'
    xml = str(i) + '.xml'
    
    command_jpg = 'copy  {} {}'.format(photos + img, destiny + img)
    command_xml = 'copy  {} {}'.format(photos + xml, destiny + xml)
    
    os.popen(command_jpg) 
    os.popen(command_xml) 